In [1]:
import os

print(os.getcwd())  # mostra onde o notebook está
print("Arquivos na pasta data/:")
print(os.listdir("data"))

C:\Users\brunn\OneDrive\Área de Trabalho\EBAC\credit_scoring_app
Arquivos na pasta data/:
['credit_scoring (1).ftr']


In [3]:
import pandas as pd

# Caminho exato do arquivo (sem os os.path, já que está na pasta correta)
df = pd.read_feather("data/credit_scoring (1).ftr")

print(df.shape)
df.head()

(750000, 15)


,data_ref,index,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda,mau
0,2015-01-01,5733,F,N,N,0,Empresário,Médio,Solteiro,Casa,43,6.873973,1.0,2515.39,False
1,2015-01-01,727,F,S,S,0,Assalariado,Médio,Casado,Casa,35,4.526027,2.0,3180.19,False
2,2015-01-01,6374,F,N,N,2,Assalariado,Médio,Casado,Casa,31,0.243836,4.0,1582.29,False
3,2015-01-01,9566,F,N,N,0,Assalariado,Médio,Casado,Casa,54,12.772603,2.0,13721.17,False
4,2015-01-01,9502,F,S,N,0,Assalariado,Superior incompleto,Solteiro,Casa,31,8.432877,1.0,2891.08,False


In [ ]:
import pandas as pd
import numpy as np
import os

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

# ============================================================
# 1. Carregar a base
# ============================================================
ARQUIVO_DADOS = "data/credit_scoring (1).ftr"   # ajuste o nome se for diferente

df = pd.read_feather(ARQUIVO_DADOS)
print("Base carregada:", df.shape)
print(df.head(3))

# garantir que data_ref é datetime
df["data_ref"] = pd.to_datetime(df["data_ref"])

# ============================================================
# 2. Separar DEV (12 primeiras safras) e OOT (3 últimas safras)
# ============================================================
# ordenar por data, só por segurança
df = df.sort_values("data_ref").reset_index(drop=True)

datas_unicas = df["data_ref"].sort_values().unique()
datas_dev = datas_unicas[:-3]   # todas menos as 3 últimas
datas_oot = datas_unicas[-3:]   # 3 últimas

df_dev = df[df["data_ref"].isin(datas_dev)].copy()
df_oot = df[df["data_ref"].isin(datas_oot)].copy()

print(f"DEV: {df_dev['data_ref'].min()} → {df_dev['data_ref'].max()} | linhas: {df_dev.shape[0]}")
print(f"OOT: {df_oot['data_ref'].min()} → {df_oot['data_ref'].max()} | linhas: {df_oot.shape[0]}")

# ============================================================
# 3. Engenharia simples: tratar tempo_emprego = 0 como "zero estrutural"
# ============================================================
for subset in [df_dev, df_oot]:
    # nova variável indicadora
    subset["tempo_emprego_zero"] = (subset["tempo_emprego"] == 0).astype(int)
    # onde é zero, vira NaN (vai ser imputado depois)
    subset.loc[subset["tempo_emprego"] == 0, "tempo_emprego"] = np.nan

# ============================================================
# 4. Definir features e target
# ============================================================
target = "mau"

# numéricas
num_cols = ["qtd_filhos", "idade", "tempo_emprego",
            "qt_pessoas_residencia", "renda"]

# categóricas originais + feature criada
cat_cols = ["sexo", "posse_de_veiculo", "posse_de_imovel",
            "tipo_renda", "educacao", "estado_civil",
            "tipo_residencia", "tempo_emprego_zero"]

# remover colunas não explicativas
drop_cols = ["index", "data_ref"]

X_dev = df_dev.drop(columns=[target] + drop_cols)
y_dev = df_dev[target].astype(int)   # True/False → 1/0

X_oot = df_oot.drop(columns=[target] + drop_cols)
y_oot = df_oot[target].astype(int)

print("X_dev:", X_dev.shape, "| y_dev:", y_dev.shape)
print("X_oot:", X_oot.shape, "| y_oot:", y_oot.shape)

# garantia de que colunas estão na ordem esperada
X_dev = X_dev[num_cols + cat_cols]
X_oot = X_oot[num_cols + cat_cols]

# ============================================================
# 5. Construir pipeline com imputação + escala + dummies + modelo
# ============================================================
from sklearn.pipeline import Pipeline

# pipeline numérico: imputa mediana e padroniza
num_pipeline = Pipeline(steps=[
    ("imp", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

# pipeline categórico: imputa valor mais frequente e faz one-hot
cat_pipeline = Pipeline(steps=[
    ("imp", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocess = ColumnTransformer(transformers=[
    ("num", num_pipeline, num_cols),
    ("cat", cat_pipeline, cat_cols)
])

modelo = LogisticRegression(max_iter=500)

pipe = Pipeline(steps=[
    ("preprocess", preprocess),
    ("modelo", modelo)
])

# ============================================================
# 6. Treinar o modelo na base de desenvolvimento
# ============================================================
pipe.fit(X_dev, y_dev)

# ============================================================
# 7. Avaliar AUC no DEV e no OOT
# ============================================================
proba_dev = pipe.predict_proba(X_dev)[:, 1]
proba_oot = pipe.predict_proba(X_oot)[:, 1]

auc_dev = roc_auc_score(y_dev, proba_dev)
auc_oot = roc_auc_score(y_oot, proba_oot)

print(f"\nAUC DEV: {auc_dev:.4f}")
print(f"AUC OOT: {auc_oot:.4f}")

# ============================================================
# 8. Salvar o modelo treinado com pickle
# ============================================================
os.makedirs("model", exist_ok=True)
modelo_path = os.path.join("model", "model_final.pkl")

import pickle
with open(modelo_path, "wb") as f:
    pickle.dump(pipe, f)

print(f"\n✅ Modelo salvo em: {modelo_path}")

Base carregada: (750000, 15)
    data_ref  index sexo posse_de_veiculo posse_de_imovel  qtd_filhos  \
0 2015-01-01   5733    F                N               N           0   
1 2015-01-01    727    F                S               S           0   
2 2015-01-01   6374    F                N               N           2   

    tipo_renda educacao estado_civil tipo_residencia  idade  tempo_emprego  \
0   Empresário    Médio     Solteiro            Casa     43       6.873973   
1  Assalariado    Médio       Casado            Casa     35       4.526027   
2  Assalariado    Médio       Casado            Casa     31       0.243836   

   qt_pessoas_residencia    renda    mau  
0                    1.0  2515.39  False  
1                    2.0  3180.19  False  
2                    4.0  1582.29  False  
DEV: 2015-01-01 00:00:00 → 2015-12-01 00:00:00 | linhas: 600000
OOT: 2016-01-01 00:00:00 → 2016-03-01 00:00:00 | linhas: 150000
X_dev: (600000, 13) | y_dev: (600000,)
X_oot: (150000, 13) | y_oo